In [1]:
import cv2
import numpy as np
import os
import import_ipynb
import random
import cvzone
import math
from hand_Track_Module import handDetector
from PIL import ImageFont, ImageDraw, Image

cam_width, cam_height = 1280, 720 #攝像頭畫面寬&高大小設定
cap = cv2.VideoCapture(0) #攝像頭設置
cap.set(3, cam_width) #設定攝像頭畫面寬度(id為3表示寬)
cap.set(4, cam_height) #設定攝像頭畫面高度(id為4表示高)
titlefontpath = "Font/王漢宗綜藝體繁.ttf"
commentfontpath = "Font/NotoSansTC[wght].ttf"

class CatYarnGame:
    def __init__(self):
        self.backgroundPath = "Background/Grass Background.jpg" #存放背景圖片資料的路徑
        self.garbagePath = "assets/P_garbage.png" #垃圾圖片的路徑
        self.foodPath = "assets/P_food.png" #食物圖片的路徑
        self.catPath = "assets/P_cat.png" #貓圖片的路徑
        
        self.limitMin_x = 0 
        self.limitMin_y = 0
        self.limitMax_x = 0 
        self.limitMax_y = 0

        self.imgGarbage = cv2.imread(self.garbagePath, cv2.IMREAD_UNCHANGED) #讀取垃圾圖片
        self.imgGarbage = cv2.resize(self.imgGarbage, (0,0), None, 0.25, 0.25) #設定垃圾圖片大小
        self.imgFood = cv2.imread(self.foodPath, cv2.IMREAD_UNCHANGED) #讀取食物圖片
        self.imgFood = cv2.resize(self.imgFood, (0,0), None, 0.25, 0.25) #設定食物圖片大小
        self.imgCat = cv2.imread(self.catPath, cv2.IMREAD_UNCHANGED) #讀取貓圖片
        self.imgCat = cv2.resize(self.imgCat, (0,0), None, 0.25, 0.25) #設定貓圖片大小

        self.garbage = [] #垃圾位置存取陣列
        self.food = [] #食物位置存取陣列
        self.cat = 5,5 #貓的位置(起始位置)

        self.pull = False #拉物件回起點的狀態(游標碰到物件後變為True，物件拉回起點後變為False)
        self.gameset = False #遊戲準備的狀態(食物&垃圾物件生成完畢後變為True，物件拉回起點後變為False)
        self.gamestart = False #遊戲開始的狀態(當游標移到起始位置變為True，游標碰到物件後變為False)
        
        self.touchGarbage = [] #紀錄抓到的垃圾座標(垃圾拉回起點後清空)
        self.touchFood = [] #紀錄抓到的食物座標(食物拉回起點後清空)
        
        self.points = [] #紀錄路線的節點
        self.lengths = [] #紀錄路線的每個節點間的長度(以利後續回溯)
        self.previousPoint = 0,0 #路線目前節點的上一個節點
        
        self.TotalScore = 0  #分數
        
        self.gameRound = 0
        self.gameRoundEnd = 5
        
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 0.5
        self.font_thickness = 1
        self.start_text = "Please place the cursor on the yarn ball to start the game"


    #攝像頭出現遊玩畫面
    def webcamPlayArea(self, gameSize_x, gameSize_y):
        offset_x, offset_y = int((cam_width - gameSize_x) / 2), int((cam_height - gameSize_y) / 2)
        self.limitMin_x, self.limitMin_y = offset_x, offset_y
        self.limitMax_x, self.limitMax_y = cam_width-offset_x, cam_height-offset_y 
        cv2.rectangle(img, (self.limitMin_x, self.limitMin_y), (self.limitMax_x, self.limitMax_y), (255,0,0), 2)  
    
    
    #垃圾&食物的座標生成
    def objectsCoordinates(self):
        overlap = 0
        self.garbage = [] #所有垃圾的座標位置清空
        for i in range(4):
            x = random.randint(100, 540)
            y = random.randint(50, 290)
            self.garbage.append([i,x,y]) 
            
        while overlap == 0:
            x = random.randint(100, 540)
            y = random.randint(50, 290)
            overlap = 1
            self.food = [x,y]
            for i in range(4):
                if x == self.garbage[i][1]:
                    if y == self.garbage[i][2]:
                        overlap = 0
        self.gameset = True  
        self.gameRound += 1
        
        
    #畫線
    def drawLine(self, img, currentPoint):
        px, py = self.previousPoint #紀錄上一個節點位置
        cx, cy = currentPoint #紀錄目前的節點位置
        
        self.points.append([cx, cy])
        distance = math.hypot(cx - px, cy - py)
        self.lengths.append(distance)
        self.previousPoint = cx, cy
        
        for index, point in enumerate(self.points):
            if index != 0:
                cv2.line(img, self.points[index - 1], self.points[index], (0, 0, 255), 5)
        
        return img
    
    
    #拉物件到起點
    def pullObject(self, img):
        totalIndex = len(self.points) - 1
        for index in range(totalIndex, 0, -1):
            cv2.line(img, self.points[index], self.points[index - 1], (0, 0, 255), 5)
            
        if self.touchFood != []:
            self.touchFood = self.points[totalIndex]
        elif self.touchGarbage != []:
            self.touchGarbage[1] = self.points[totalIndex][0]
            self.touchGarbage[2] = self.points[totalIndex][1]
            
        self.points.pop(totalIndex)
        
        if self.points == []:
            self.updateScore()  # 更新分數
            self.gameset = False
            self.pull = False
            self.touchFood = []
            self.touchGarbage = []
        return img
    
    #偵測碰撞
    def detectCollision(self, gamePoint):
        if gamePoint[0] >= self.cat[0] and gamePoint[0] <= self.cat[0] + self.imgCat.shape[1]:
            if gamePoint[1] >= self.cat[1] and gamePoint[1] <= self.cat[1] + self.imgCat.shape[0]:
                self.gamestart = True
        if self.gamestart == True:
            for i in range(4):
                if gamePoint[0] >= (self.garbage[i][1]) and gamePoint[0] <= (self.garbage[i][1]) + self.imgGarbage.shape[1]:
                    if gamePoint[1] >= (self.garbage[i][2]) and gamePoint[1] <= (self.garbage[i][2]) + self.imgGarbage.shape[0]:
                        self.gamestart = False
                        self.pull = True
                        self.touchGarbage = self.garbage[i]
            
            if gamePoint[0] >= (self.food[0]) and gamePoint[0] <= (self.food[0]) + self.imgFood.shape[1]:
                if gamePoint[1] >= (self.food[1]) and gamePoint[1] <= (self.food[1]) + self.imgFood.shape[0]:
                    self.gamestart = False
                    self.pull = True
                    self.touchFood = self.food
    
    #更新分數
    def updateScore(self):
        if self.touchFood:
            self.TotalScore += 100 #碰到食物+100
        elif self.touchGarbage:
            self.TotalScore -= 10000 #碰到垃圾+10000
                        
    #物件出現在遊戲畫面
    def objectsAppear(self, img):
        img = cvzone.overlayPNG(img, self.imgCat, (self.cat[0], self.cat[1]))
        if self.touchFood == []:
            img = cvzone.overlayPNG(img, self.imgFood, (self.food[0], self.food[1]))
        elif self.touchFood != []:
            img = cvzone.overlayPNG(img, self.imgFood, (self.touchFood[0], self.touchFood[1]))
            
        if self.touchGarbage == []:
            for i in range(4):
                img = cvzone.overlayPNG(img, self.imgGarbage, (self.garbage[i][1], self.garbage[i][2]))
        elif self.touchGarbage != []:
            for i in range(4):
                if i == self.touchGarbage[0]:
                    img = cvzone.overlayPNG(img, self.imgGarbage, (self.touchGarbage[1], self.touchGarbage[2]))
                    continue
                img = cvzone.overlayPNG(img, self.imgGarbage, (self.garbage[i][1], self.garbage[i][2]))
        
        # 在貓咪底下顯示分數
        cv2.putText(img, f"Score: {self.TotalScore}", (self.cat[0], self.cat[1] + self.imgCat.shape[0] + 20),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        
        if self.pull == False and self.gamestart == False:
            cv2.putText(img, self.start_text, ( 50, 190),
                    self.font, self.font_scale, (0, 0, 0), self.font_thickness, cv2.LINE_AA)
        
        return img


detector = handDetector() #建立偵測class
game = CatYarnGame()

while True:
    #第一步驟
    retval, img = cap.read() #讀取攝像頭照到的影像，retval表示影像捕獲成功or失敗
    img = detector.findHands(img) #偵測手掌
    landmarkList = detector.findPosition(img, 0) #獲取手掌偵測節點資料
    img = np.flip(img, axis=1) #攝像頭畫面左右翻轉(axis=1表示左右翻轉，axis=0表示上下翻轉)
    img = np.ascontiguousarray(img)
    imgBackground = cv2.imread(game.backgroundPath)
    
    
    #第二步驟
    gameSize_x, gameSize_y = imgBackground.shape[1], imgBackground.shape[0] #獲取背景圖片大小(作為遊玩畫面大小)
    game.webcamPlayArea(gameSize_x, gameSize_y) #攝像頭繪製遊玩範圍
    
    if game.gameRound < game.gameRoundEnd + 1:
        #第三步驟
        if game.gameset == False:
            game.objectsCoordinates() #生成物件座標並改變遊戲準備狀態


        #第四步驟
        if len(landmarkList) != 0: #如果有偵測到手部的時候
            finger_x, finger_y = -(landmarkList[8][1] - cam_width), landmarkList[8][2] #獲取攝像頭的食指座標
            gamePoint = (finger_x - game.limitMin_x, finger_y - game.limitMin_y) #將攝像頭的食指座標轉換為遊戲座標

            if finger_x > game.limitMin_x and finger_x < game.limitMax_x and finger_y > game.limitMin_y and finger_y < game.limitMax_y: #若食指座標有在遊戲範圍內
                cv2.circle(imgBackground, gamePoint, 5, (0, 0, 255), cv2.FILLED) #在遊戲畫面顯示座標

            if game.gamestart == True:
                imgBackground = game.drawLine(imgBackground, gamePoint) 

            game.detectCollision(gamePoint) #偵測物體碰撞，並改變遊戲開始&準備的狀態

        if game.pull == True:
            imgBackground = game.pullObject(imgBackground)   


        #第五步驟
        imgBackground = game.objectsAppear(imgBackground)
        
    elif game.gameRound == game.gameRoundEnd + 1:
        imgPil = Image.fromarray(imgBackground)
        draw = ImageDraw.Draw(imgPil)
        if game.TotalScore == 500:
            text = "你的貓貓吃得很開心(✪ω✪)"
        elif game.TotalScore == -50000:
            text = "你的貓是天生的垃圾車∑(✘Д✘)"
        elif game.TotalScore < 0:
            text = "你在餵什麼東西！！╰(‵□′)╯"
        draw.text((gameSize_x/6 - 10, gameSize_y/2 - 40), text, fill=(0, 0, 0), font=ImageFont.truetype(commentfontpath, 30))
        imgBackground = np.array(imgPil)
        #印總分
        #印Press esc to quit
    
    #第六步驟
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL) #新增攝像頭視窗(Image)，允許視窗自由縮放
    cv2.resizeWindow("Image", img.shape[1]//2, img.shape[0]//2) #改變"Image"視窗大小
    cv2.imshow("Image", img) #攝像頭照到的影像顯示到"Image"視窗
    
    cv2.namedWindow("Game", cv2.WINDOW_NORMAL) #新增遊戲視窗(Game)，允許視窗自由縮放
    cv2.resizeWindow("Game", imgBackground.shape[1]*2, imgBackground.shape[0]*2) #改變"Game"視窗大小
    cv2.imshow("Game", imgBackground) #將背景圖片顯示到"Game"視窗
    
    key = cv2.waitKey(1) #等待1毫秒接收按鍵
    if key == 27: #若按下Esc按鍵
        break #退出迴圈
    
cap.release() #結束攝像機
cv2.destroyAllWindows() #關閉所有視窗

importing Jupyter notebook from hand_Track_Module.ipynb
